In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 49.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=b9d3cc09304240c233f10b4ae506c8aff03cfb65f31b884607f8834027f89dca
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import numpy as np
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
#create instance of spark class
spark=SparkSession.builder.appName('boston').getOrCreate()
  
#create spark dataframe of input csv file
df=spark.read.csv('/content/boston.csv',inferSchema=True,header=True)
df.show(10)

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|18.0|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|       24.0|
|0.027310001| 0.0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729| 0.0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001| 0.0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999| 0.0|2.

In [ ]:
df.cache()
df.printSchema()

root
 |-- CRIM: double (nullable = true)
 |-- ZN: double (nullable = true)
 |-- INDUS: double (nullable = true)
 |-- CHAS: integer (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: integer (nullable = true)
 |-- TAX: integer (nullable = true)
 |-- PT: double (nullable = true)
 |-- B: double (nullable = true)
 |-- LSTAT: double (nullable = true)
 |-- MV: double (nullable = true)



In [ ]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
CRIM,506,3.6135235608162057,8.601545086715594,0.00632,88.97619629
ZN,506,11.363636363636363,23.32245299451514,0.0,100.0
INDUS,506,11.136778749531626,6.86035298095724,0.460000008,27.73999977
CHAS,506,0.0691699604743083,0.2539940413404101,0,1
NOX,506,0.5546950602312246,0.1158776754570543,0.38499999,0.870999992
RM,506,6.28463438896641,0.7026171549511354,3.561000109,8.779999733
AGE,506,68.57490120115612,28.148861532793276,2.900000095,100.0
DIS,506,3.7950426960059325,2.105710142043288,1.129600048,12.12650013
RAD,506,9.549407114624506,8.707259384239366,1,24


In [ ]:
df.columns

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PT',
 'B',
 'LSTAT',
 'MV']

In [ ]:
print(df.count(),len(df.columns))

506 14


In [ ]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+----+---+-----+----+---+---+---+---+---+---+---+---+-----+---+
|CRIM| ZN|INDUS|CHAS|NOX| RM|AGE|DIS|RAD|TAX| PT|  B|LSTAT| MV|
+----+---+-----+----+---+---+---+---+---+---+---+---+-----+---+
|   0|  0|    0|   0|  0|  0|  0|  0|  0|  0|  0|  0|    0|  0|
+----+---+-----+----+---+---+---+---+---+---+---+---+-----+---+



In [ ]:
import six
for i in df.columns:
    if not( isinstance(df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to MV for ", i, df.stat.corr('MV',i))

Correlation to MV for  CRIM -0.3883046116575088
Correlation to MV for  ZN 0.36044534463752903
Correlation to MV for  INDUS -0.48372517128143383
Correlation to MV for  CHAS 0.17526017775291847
Correlation to MV for  NOX -0.4273207763683772
Correlation to MV for  RM 0.695359937127267
Correlation to MV for  AGE -0.37695456714288667
Correlation to MV for  DIS 0.24992873873512172
Correlation to MV for  RAD -0.3816262315669168
Correlation to MV for  TAX -0.46853593528654536
Correlation to MV for  PT -0.5077867038116085
Correlation to MV for  B 0.3334608226834164
Correlation to MV for  LSTAT -0.7376627294671615
Correlation to MV for  MV 1.0


In [ ]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT'], outputCol = 'features')
v_df = vectorAssembler.transform(df)
v_df = v_df.select(['features', 'MV'])
v_df.show(3)

+--------------------+-----------+
|            features|         MV|
+--------------------+-----------+
|[0.00632,18.0,2.3...|       24.0|
|[0.027310001,0.0,...|21.60000038|
|[0.02729,0.0,7.07...|34.70000076|
+--------------------+-----------+
only showing top 3 rows



In [ ]:
splits = v_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [ ]:
train_df.describe().show()

+-------+-----------------+
|summary|               MV|
+-------+-----------------+
|  count|              366|
|   mean|22.33961753109836|
| stddev|9.134168239125092|
|    min|              5.0|
|    max|             50.0|
+-------+-----------------+



In [ ]:
test_df.describe().show()

+-------+-----------------+
|summary|               MV|
+-------+-----------------+
|  count|              140|
|   mean|23.03785716462857|
| stddev|9.373876665300825|
|    min|      7.199999809|
|    max|             50.0|
+-------+-----------------+



In [ ]:
from pyspark.ml.regression import LinearRegression
LR = LinearRegression(featuresCol = 'features', labelCol='MV', maxIter=10, regParam=0.3, elasticNetParam=0.8)
LR_model = LR.fit(train_df)

In [ ]:
print("Coefficients: " + str(LR_model.coefficients))
print("Intercept: " + str(LR_model.intercept))

Coefficients: [-0.051655374225279105,0.0,0.0,2.8282666508924383,-5.0651388455090105,4.726096899791141,0.0,-0.5449872757567288,0.0,-0.0006534053015874601,-0.7724740682301656,0.0046111685826062026,-0.5509693797441347]
Intercept: 17.483235980389253


In [ ]:
trainingSummary = LR_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 4.499588
r2: 0.756670


In [ ]:
lr_predictions = LR_model.transform(test_df)
lr_predictions.select("prediction","MV","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="MV",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
| 38.37597844891789|       50.0|[0.01381,80.0,0.4...|
|30.880659271248753|31.60000038|[0.01432,100.0,1....|
| 27.27677663733315|       24.5|[0.01501,80.0,2.0...|
|39.267181963981784|       50.0|[0.020090001,95.0...|
| 25.64778123072346|24.70000076|[0.02055,85.0,0.7...|
+------------------+-----------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.602166


In [ ]:
test_result = LR_model.evaluate(test_df)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)

Root Mean Squared Error (RMSE) on test data = 5.89134


In [ ]:
trainingSummary.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -6.6770369532887095|
|   1.737162947712065|
|  -5.298216626713433|
|  0.4336499403969185|
|    5.23817111264594|
|  2.5858820571108367|
| -1.2729461815264074|
|   8.035054340933456|
|   9.111794959749744|
|   3.466759975876453|
|  1.9837077409183763|
| -2.6017062581152253|
|   6.208707843655354|
|-0.15686924375699718|
|   5.844497102937822|
| 0.08238678873582472|
| -10.325708331852812|
|   2.660888387789065|
|   -4.03168710192455|
| 0.17257798255677415|
+--------------------+
only showing top 20 rows



/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
predictions = LR_model.transform(test_df)
predictions.select("prediction","MV","features").show()

+------------------+-----------+--------------------+
|        prediction|         MV|            features|
+------------------+-----------+--------------------+
| 38.37597844891789|       50.0|[0.01381,80.0,0.4...|
|30.880659271248753|31.60000038|[0.01432,100.0,1....|
| 27.27677663733315|       24.5|[0.01501,80.0,2.0...|
|39.267181963981784|       50.0|[0.020090001,95.0...|
| 25.64778123072346|24.70000076|[0.02055,85.0,0.7...|
| 27.58932781284717|23.89999962|[0.025429999,55.0...|
| 28.68220233741542|30.79999924|[0.027629999,75.0...|
| 26.65802181704686|       25.0|[0.028750001,28.0...|
|19.925315434528027|       18.5|[0.030409999,0.0,...|
| 30.15690208874083|34.90000153|[0.031500001,95.0...|
|31.998614103832715|34.90000153|[0.03359,75.0,2.9...|
| 37.90886350443603|       48.5|[0.035100002,95.0...|
|27.379134119262226|       22.0|[0.03537,34.0,6.0...|
|24.329091049009538|22.89999962|[0.03551,25.0,4.8...|
| 37.70087264272081|45.40000153|[0.035780001,20.0...|
|28.194112810753985|27.89999

In [ ]:
#evaluating model trained for Rsquared error
train_results=LR_model.evaluate(train_df)
  
print('Rsquared Error :',train_results.r2)

Rsquared Error : 0.7566700184912659


In [ ]:
test_results=LR_model.evaluate(test_df)
print('Rsquared Error :',test_results.r2)

Rsquared Error : 0.6021656045323877
